In [1]:
import pandas as pd

In [2]:
# bidmc
ICDs = pd.read_csv('/home/bram/Project_PD/PD project/Cohort construction/initial classification/MEDICDDATA POS/BIDMC_ICDPLUS.csv')

In [3]:
matrix=pd.read_csv('/home/bram/Project_PD/PD project/Feature matrix/01_Text_feature_matrix/feat_matrix_dropped.csv')

In [4]:
print(len(matrix))
print(matrix.columns)

3827
Index(['BDSPPatientID', 'CreateDate', 'hospital', 'parkinson disease_pos',
       'pd_pos', 'parkinson_pos', 'parkinsonismx_pos', 'disease_pos',
       'akathisia_pos', 'anosmia_pos',
       ...
       'exelon patch_neg', 'exelon_neg', 'cane_neg', 'fall_neg',
       'deep brain stimulation_neg', 'dbs_neg', 'bradykinesia_neg',
       'hypokinesia_neg', 'dementia_neg', 'pneumonia_neg'],
      dtype='object', length=211)


In [5]:
print(ICDs.head())
print(len(ICDs))

   BDSPPatientID  DiagnosisSequenceNumber DiagnosisCode  CreateDate  \
0      150072288                        5          3320  2016-05-14   
1      150072288                        5          3320  2016-05-14   
2      150070124                        3          3320  2016-05-09   
3      150073126                        1          3320  2016-09-06   
4      150000169                        2          3320  2016-05-19   

  DischargeDate  
0    2016-05-15  
1    2016-05-14  
2    2016-05-09  
3    2016-09-06  
4    2016-05-19  
32128


In [6]:
# import BIDMC 

# # example
# ICDs = pd.read_csv('/home/cdac-c-15/Desktop/NAXCA/Cohort All Files/allICDCodesMGB.csv')
# ICDs = ICDs[ICDs['BDSPPatientID'].isin(set(matrix['BDSPPatientID']))]
# ICDs['ICD Date'] = pd.to_datetime(ICDs['ShiftedContactDTS']).dt.strftime('%Y-%m-%d')
# ICDs['ICD Date'] = pd.to_datetime(ICDs['ICD Date'])
# ICDs = ICDs[['BDSPPatientID', 'ICD Date']]
# ICDs.head()

ICDs = ICDs[ICDs['BDSPPatientID'].isin(set(matrix['BDSPPatientID']))]
ICDs['ICD Date'] = pd.to_datetime(ICDs['CreateDate']).dt.strftime('%Y-%m-%d')
# ICDs['ICD Date'] = pd.to_datetime(ICDs['ICD Date'])
ICDs = ICDs[['BDSPPatientID', 'ICD Date']]
ICDs.head()

,BDSPPatientID,ICD Date
8,150001276,2016-05-18
9,150287462,2016-06-02
12,150001629,2016-05-02
13,150001629,2016-07-27
14,150001629,2016-08-10


In [7]:
print(ICDs['BDSPPatientID'].nunique())

500


In [8]:
print(len(ICDs))

3278


In [9]:
ICDs_MGB = pd.read_csv('/home/bram/Project_PD/PD project/Cohort construction/initial classification/MEDICDDATA POS/MGB_ICDPLUS.csv')
print(ICDs_MGB.head())
print(len(ICDs_MGB))

ICDs_MGB = ICDs_MGB[ICDs_MGB['BDSPPatientID'].isin(set(matrix['BDSPPatientID']))]
ICDs_MGB['ICD Date'] = pd.to_datetime(ICDs_MGB['CreateDate']).dt.strftime('%Y-%m-%d')
# ICDs['ICD Date'] = pd.to_datetime(ICDs['ICD Date'])
ICDs_MGB = ICDs_MGB[['BDSPPatientID', 'ICD Date']]
ICDs_MGB.head()
print(len(ICDs_MGB))

   BDSPEncounterID  EncounterLineNBR  BDSPPatientID  CreateDate  ICDLineNBR  \
0     1.339649e+10                 1    113319054.0  2017-09-21         1.0   
1     1.347001e+10                 1    115367785.0  2020-02-18         1.0   
2     1.338726e+10                 1    117147113.0  2017-09-14         1.0   
3     1.354173e+10                 1    112059081.0  2022-05-31         1.0   
4     1.339986e+10                 4    119027616.0  2019-09-29         1.0   

   ICDCD             ICDDSC                                  DiagnosisNM  \
0  332.0  Paralysis agitans  Parkinsonism, unspecified Parkinsonism type   
1  332.0  Paralysis agitans                            Parkinson disease   
2  332.0  Paralysis agitans                          Parkinson's disease   
3  332.0  Paralysis agitans                            Parkinson disease   
4  332.0  Paralysis agitans                          Parkinson's disease   

  DiagnosisDSC PrimaryDiagnosisFLG DiagnosisChronicFLG  \
0         

/tmp/ipykernel_549360/216046602.py:1: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  ICDs_MGB = pd.read_csv('/home/bram/Project_PD/PD project/Cohort construction/initial classification/MEDICDDATA POS/MGB_ICDPLUS.csv')


In [10]:
print(ICDs_MGB.head())
print(len(ICDs_MGB))
print(ICDs_MGB['BDSPPatientID'].nunique()) #validated 964

    BDSPPatientID    ICD Date
13    113996673.0  2017-10-01
17    121888695.0  2021-07-07
35    114337378.0  2021-04-03
38    120578313.0  2017-11-08
45    113580635.0  2022-06-12
33587
964


In [11]:
ICDs_STF = pd.read_csv('/home/bram/Project_PD/PD project/Cohort construction/initial classification/MEDICDDATA POS/STF_ICDPLUS.csv')
print(ICDs_STF.head())
print(len(ICDs_STF))


   bdsp_patient_id           CreateDate     age      type  \
0        177501743  2016-05-01 00:00:00  73.540  Primary    
1        177501743  2016-05-01 00:00:00  73.540  Primary    
2        177501743  2016-05-01 08:17:00  73.541       NaN   
3        177501743  2016-05-01 08:17:00  73.541       NaN   
4        177511104  2016-05-02 00:00:00  62.099       NaN   

                      source icd9_code icd10_code           description  \
0  Professional Billing Code       NaN        G20   Parkinson's disease   
1        Encounter Diagnosis     332.0     G20.A1  Parkinson's syndrome   
2            Admit Diagnosis       NaN        G20   Parkinson's disease   
3  Professional Billing Code       NaN        G20   Parkinson's disease   
4        Encounter Diagnosis     332.0     G20.A1   Parkinson's disease   

  performing_provider billing_provider  bdsp_patient_encounter_id  
0           NEUROLOGY        NEUROLOGY               2.314206e+11  
1           NEUROLOGY              NaN        

In [12]:
ICDs_STF['BDSPPatientID']=ICDs_STF['bdsp_patient_id']
print(ICDs_STF.head())

   bdsp_patient_id           CreateDate     age      type  \
0        177501743  2016-05-01 00:00:00  73.540  Primary    
1        177501743  2016-05-01 00:00:00  73.540  Primary    
2        177501743  2016-05-01 08:17:00  73.541       NaN   
3        177501743  2016-05-01 08:17:00  73.541       NaN   
4        177511104  2016-05-02 00:00:00  62.099       NaN   

                      source icd9_code icd10_code           description  \
0  Professional Billing Code       NaN        G20   Parkinson's disease   
1        Encounter Diagnosis     332.0     G20.A1  Parkinson's syndrome   
2            Admit Diagnosis       NaN        G20   Parkinson's disease   
3  Professional Billing Code       NaN        G20   Parkinson's disease   
4        Encounter Diagnosis     332.0     G20.A1   Parkinson's disease   

  performing_provider billing_provider  bdsp_patient_encounter_id  \
0           NEUROLOGY        NEUROLOGY               2.314206e+11   
1           NEUROLOGY              NaN      

In [13]:
ICDs_STF = ICDs_STF[ICDs_STF['BDSPPatientID'].isin(set(matrix['BDSPPatientID']))]
ICDs_STF['ICD Date'] = pd.to_datetime(ICDs_STF['CreateDate']).dt.strftime('%Y-%m-%d')
# ICDs['ICD Date'] = pd.to_datetime(ICDs['ICD Date'])
ICDs_STF = ICDs_STF[['BDSPPatientID', 'ICD Date']]
ICDs_STF.head()
print(len(ICDs_STF))

20242


In [14]:
print(ICDs_STF['BDSPPatientID'].nunique())#validated icd+ are 363 patients
print(ICDs_STF.head())

363
   BDSPPatientID    ICD Date
4      177511104  2016-05-02
5      177511104  2016-05-02
6      177527333  2016-05-03
7      177527333  2016-05-03
8      177527333  2016-05-03


In [15]:
ICDs_compl=[ICDs,ICDs_STF ,ICDs_MGB]

ICDs_complete=pd.concat(ICDs_compl, ignore_index=True)

In [16]:
print(ICDs_complete.head)
print(len(ICDs_complete))
print(ICDs_complete['BDSPPatientID'].nunique())

<bound method NDFrame.head of        BDSPPatientID    ICD Date
0        150001276.0  2016-05-18
1        150287462.0  2016-06-02
2        150001629.0  2016-05-02
3        150001629.0  2016-07-27
4        150001629.0  2016-08-10
...              ...         ...
57102    116218371.0  2017-02-11
57103    115108029.0  2022-08-10
57104    112639696.0  2017-01-12
57105    122217727.0  2020-03-20
57106    117270174.0  2022-06-26

[57107 rows x 2 columns]>
57107
1827


In [17]:
matrix['CreateDate']= pd.to_datetime(matrix['CreateDate']) 
ICDs_complete['ICD Date'] = pd.to_datetime(ICDs_complete['ICD Date'])

In [18]:
# add 1 or 0 to new column in matrix that denotes whether an icd is present +- 6 months from note creation date
#merge the dfs on patient ID
merged_df = pd.merge(matrix, ICDs_complete, on='BDSPPatientID')
#check for date timeline
merged_df['ICD'] = merged_df.apply(
    lambda row: (row['CreateDate'] >= row['ICD Date'] - pd.DateOffset(months=6)) and 
                (row['CreateDate'] <= row['ICD Date'] + pd.DateOffset(months=6)), axis=1)
matrix['ICD'] = matrix.apply(
    lambda row: merged_df[(merged_df['BDSPPatientID'] == row['BDSPPatientID']) & merged_df['ICD']].shape[0] > 0, axis=1).astype(int)

print(len(matrix))
matrix.head()

3827


,BDSPPatientID,CreateDate,hospital,parkinson disease_pos,pd_pos,parkinson_pos,parkinsonismx_pos,disease_pos,akathisia_pos,anosmia_pos,...,exelon_neg,cane_neg,fall_neg,deep brain stimulation_neg,dbs_neg,bradykinesia_neg,hypokinesia_neg,dementia_neg,pneumonia_neg,ICD
0,150709433,2017-10-23,BIDMC,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,150669096,2022-03-09,BIDMC,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,151275717,2018-05-01,BIDMC,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
3,151013494,2021-06-09,BIDMC,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,150642824,2021-11-24,BIDMC,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
print(matrix.sample(20))

      BDSPPatientID CreateDate hospital  parkinson disease_pos  pd_pos  \
599       151249469 2016-07-16    BIDMC                      0       0   
2207      115567637 2019-09-18      MGB                      0       0   
2803      111377874 2016-05-16      MGB                      0       1   
1963      116850554 2019-03-04      MGB                      0       0   
3724      121924461 2022-06-10      MGB                      1       0   
2865      112293874 2016-05-27      MGB                      0       0   
3771      118461281 2021-10-30      MGB                      0       1   
3817      114712307 2021-12-13      MGB                      0       1   
3537      114649521 2017-06-16      MGB                      0       0   
982       151302721 2022-01-16    BIDMC                      1       1   
1949      120424010 2016-11-21      MGB                      0       0   
1056      177502501 2020-08-05      STF                      1       0   
564       150365329 2017-06-07    BIDM

In [20]:
matrix.to_csv('matrix_incl_icd.csv', index=False)